In [ ]:
# basic_example.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to interact with the fledgeling package

In [ ]:
# basic imports
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# sloth
import sys
sys.path.append("../")
from fledgeling import Fledgeling, config

In [ ]:
# Setting everything up
config['experimental data']['filepath'] =  "/home/unimelb.edu.au/smeighenberg/snap/firefox/common/Downloads/icecube_10year_ps"
fledge = Fledgeling()
fledge.close()

In [ ]:
dec_range = [-5, 90]
dec_range2 = [-90, -5]
years = range(4, 10)
ebins = np.arange(0, 10, 0.1)

In [ ]:
# The Astrophysical flux from an IceCube measurement (best fit)
def astro_flux(E):
    return 1.66 * 1e-18 * (E / 1e5)**(-2.53)

In [ ]:
def count_weighting(flux_dic, conversion_tables, cuts=[31, 101]):
    """ weighting function for atmospheric events
    """
    tmp_flux = flux_dic
    tmp_flux[85] = tmp_flux[80]
    tmp_flux[86] = tmp_flux[80]
    tmp_flux[87] = tmp_flux[80]
    tmp_flux[88] = tmp_flux[80]
    tmp_flux[89] = tmp_flux[80]
    tmp_flux[90] = tmp_flux[80]
    # Example shower set only has every tenth value
    weighted_counts = {}
    # Assume up-going is produced by 0 degree fluxes
    for i in range(91, 181):
        tmp_flux[i] = tmp_flux[0]
    for year in range(4, 10):
        weighted_counts[year] = np.array([
            np.sum(
                (tmp_flux[i]["numu"] * tmp_flux[i]["e width"])[cuts[0]:cuts[1]][np.newaxis].T *
                conversion_tables[year][i],
                axis=0
            ) / 100 for i in range(85, 180)
        ])
    return weighted_counts
def count_weighting_astro(conversion_tables):
    """ weighting function for astrophyisical events
    """
    weighted_counts = {}
    for year in range(4, 10):
        weighted_counts[year] = np.array([
            np.sum(
                (astro_flux(np.sqrt(np.logspace(2, 9, 71)[1:]*np.logspace(2, 9, 71)[:-1])) * (np.logspace(2, 9, 71)[1:] - np.logspace(2, 9, 71)[:-1]))[np.newaxis].T *
                conversion_tables[year][i],
                axis=0
            ) / 100 for i in range(85, 180)
        ])
    return weighted_counts

In [ ]:
weighted_counts = count_weighting(fledge._atmos.cascade, fledge._dr.conversion_tables)
weighted_counts_astro = count_weighting_astro(fledge._dr.conversion_tables)

In [ ]:
# Atmos
up_going = np.array([
    np.trapz(weighted_counts[i], axis=0) * fledge._dr._uptime_tot_dic[i]
    for i in weighted_counts.keys()
])
total_up = np.sum(up_going, axis=0)
# Astro
astro = np.array([
    np.trapz(weighted_counts_astro[i], axis=0) * fledge._dr._uptime_tot_dic[i]
    for i in weighted_counts_astro.keys()
])
astro_tot = np.sum(astro, axis=0)

In [ ]:
tmp = fledge._dr._event_dic[fledge._dr._event_dic["dec"].between(dec_range[0], dec_range[1])]
total = np.zeros(len(ebins)-1)
for i in range(4, 10):
    yearly_events = tmp[tmp["year"] == i]
    counts, _ = np.histogram(yearly_events["E"].values, ebins)
    total += counts
local_bins = np.linspace(2, 9, 71)
# -------------------------------------------------------
# Plotting
fig = plt.figure(figsize=(4, 3), dpi=500)
ax = fig.add_subplot(111)
ax.errorbar((ebins[1:] + ebins[:-1]) / 2, total, fmt="o", color="k", yerr=np.max([np.sqrt(total), 0.2 * total], axis=0), markersize=1, label='Data')
# Note that a high agreement for energies above 1 TeV can be reached by simply shifting the energy grid of the simulation by 100 GeV!
ax.step((local_bins[1:] + local_bins[:-1]) / 2 + 0.1, total_up, color='r', where="mid", label='Atmospherics')
ax.step((local_bins[1:] + local_bins[:-1]) / 2 + 0.1, astro_tot, color='g', where="mid", label='Astrophysical')
ax.step((local_bins[1:] + local_bins[:-1]) / 2 + 0.1, (astro_tot + total_up), color='k', where="mid", label='Total')
ax.set_xlim(2.3, 6.)
ax.set_ylim(1, 1e6)
ax.set_xlabel(r"$E_\mathrm{reco}$ [log$_{10}$(E/GeV)]")
ax.set_ylabel(r"Counts")
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
plt.legend(frameon=False)
plt.yscale("log")
plt.tick_params(axis='both', which='both', direction='in')
plt.tight_layout()
fig.savefig('../pics/model_vs_data', dpi=500)